# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
hv.extension("bokeh")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df["Max Temp"] = (city_data_df["Max Temp"] - 273.15)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.7826,147.0587,10.72,95,0,0.96,AU,1707493672
1,1,edinburgh of the seven seas,-37.0676,-12.3116,19.72,89,92,11.42,SH,1707493652
2,2,waitangi,-43.9535,-176.5597,12.86,61,9,2.54,NZ,1707493663
3,3,antofagasta,-23.6500,-70.4000,23.60,66,29,4.19,CL,1707493918
4,4,kuwana,35.0667,136.7000,7.00,63,0,2.12,JP,1707493918


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600, 
    size = "Humidity",
    color = "City", 
    alpha = .75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,margaret river,-33.9500,115.0667,22.19,54,0,2.97,AU,1707493674
74,74,geraldton,-28.7667,114.6000,24.23,33,0,2.06,AU,1707493655
114,114,pisco,-13.7000,-76.2167,26.12,83,0,4.12,PE,1707493662
269,269,haiku-pauwela,20.9219,-156.3051,21.69,84,0,3.13,US,1707493672
287,287,al hazm,16.1641,44.7769,24.61,28,0,3.64,YE,1707493955
320,320,acapulco de juarez,16.8634,-99.8901,25.90,78,0,1.54,MX,1707493666
401,401,la vega,19.2221,-70.5296,24.47,43,0,3.03,DO,1707493973
420,420,salalah,17.0151,54.0924,25.05,57,0,2.06,OM,1707493975
431,431,idri,27.5000,13.2667,21.49,16,0,2.03,LY,1707493976
513,513,ghat,24.9647,10.1728,24.36,9,0,1.51,LY,1707493984


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

data = {
    "City": city, 
    "Country": country, 
    "Lat": lat,
    "Lng": lng, 
    "Humidity": humidity
}

hotel_df = pd.DataFrame(data)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
40,margaret river,AU,-33.9500,115.0667,54,
74,geraldton,AU,-28.7667,114.6000,33,
114,pisco,PE,-13.7000,-76.2167,83,
269,haiku-pauwela,US,20.9219,-156.3051,84,
287,al hazm,YE,16.1641,44.7769,28,
320,acapulco de juarez,MX,16.8634,-99.8901,78,
401,la vega,DO,19.2221,-70.5296,43,
420,salalah,OM,17.0151,54.0924,57,
431,idri,LY,27.5000,13.2667,16,
513,ghat,LY,24.9647,10.1728,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
geraldton - nearest hotel: Ocean Centre Hotel
pisco - nearest hotel: La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600, 
    size = "Humidity",
    color = "City", 
    alpha = .75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2